In [3]:
import os
import csv
from supabase import create_client, Client
import os
import datetime
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
supabase: Client = create_client(url, key)

data = supabase.table("league").select("*").execute()
print(data)

data=[{'league_unique': 'ncaambb', 'league_name': "NCAA Men's Basketball", 'sport': 'basketball', 'womens': False}] count=None


In [4]:
COMPETITION_ID = 6
POOL_ID = 16
TODAY = datetime.date.today()
# TODAY = "2025-03-20"
CURRENT_ROUND = 2
YEAR=2025


In [16]:
def update_game_schedule(game_schedule_csv):
    game_insert = []
    with open(game_schedule_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        # next(reader)
        for row in reader:
            print(row)
            if row != {'game_date': '', 'team_2_id': '', 'team_1_id': '', 'game_time': '', 'round_num': '', 'competition_id': ''}:
                game_insert.append(row)

    data = supabase.table("game").upsert(game_insert,
                                         # ignore_duplicates=True, on_conflict="game_date, team_1_id, team_2_id"
                                         ).execute()
    print(data)

In [18]:
# round = CURRENT_ROUND
round = 3
update_game_schedule(f"{YEAR}/schedules/game-schedule-round-{round}-{YEAR}-ncaa-tournament.csv")

{'game_date': '2025-03-22', 'team_2_id': 'mcneese-state', 'team_1_id': 'purdue', 'game_time': '12:45', 'round_num': '3', 'competition_id': '6'}
{'game_date': '2025-03-22', 'team_2_id': 'arkansas', 'team_1_id': 'st-johns-ny', 'game_time': '15:15', 'round_num': '3', 'competition_id': '6'}
{'game_date': '2025-03-22', 'team_2_id': 'michigan', 'team_1_id': 'texas-am', 'game_time': '17:30', 'round_num': '3', 'competition_id': '6'}
{'game_date': '2025-03-22', 'team_2_id': 'drake', 'team_1_id': 'texas-tech', 'game_time': '18:10', 'round_num': '3', 'competition_id': '6'}
{'game_date': '2025-03-22', 'team_2_id': 'creighton', 'team_1_id': 'auburn', 'game_time': '19:10', 'round_num': '3', 'competition_id': '6'}
{'game_date': '2025-03-22', 'team_2_id': 'brigham-young', 'team_1_id': 'wisconsin', 'game_time': '20:00', 'round_num': '3', 'competition_id': '6'}
{'game_date': '2025-03-22', 'team_2_id': 'gonzaga', 'team_1_id': 'houston', 'game_time': '20:40', 'round_num': '3', 'competition_id': '6'}
{'gam

In [13]:
import pandas as pd
import datetime
def generate_game_scoring_sheet(competition_id: int = COMPETITION_ID, date = datetime.date.today()):
    data = supabase.table("players_in_games_view").select("*").eq("game_date", date).eq('competition_id', competition_id).execute()
    print(data)
    df = pd.DataFrame(data.data)
    df['points'] = None
    df['lost'] = None
    df['inactive'] = None
    # df['rebounds'] = 0
    # df['assists'] = 0
    df.to_csv(f"{YEAR}/scores/{date}-game-scoring-{YEAR}-ncaa-tournament.csv", index=False, columns=['game_time', 'team_unique', 'lost', 'player_unique', 'points','inactive','game_id'])
    # 'rebounds', 'assists'])

In [20]:
# date = datetime.date.today()
# date = "2024-03-28"
# date = TODAY
date = "2025-03-23"
# date= "2024-03-31"
generate_game_scoring_sheet(competition_id=COMPETITION_ID, date=date)

data=[{'player_unique': 'solomon-ball-2', 'team_unique': 'connecticut', 'game_id': 430, 'game_date': '2025-03-23', 'game_time': '12:10:00', 'round_num': 3, 'competition_id': 6}, {'player_unique': 'alex-karaban-1', 'team_unique': 'connecticut', 'game_id': 430, 'game_date': '2025-03-23', 'game_time': '12:10:00', 'round_num': 3, 'competition_id': 6}, {'player_unique': 'liam-mcneeley-1', 'team_unique': 'connecticut', 'game_id': 430, 'game_date': '2025-03-23', 'game_time': '12:10:00', 'round_num': 3, 'competition_id': 6}, {'player_unique': 'tarris-reed-jr-1', 'team_unique': 'connecticut', 'game_id': 430, 'game_date': '2025-03-23', 'game_time': '12:10:00', 'round_num': 3, 'competition_id': 6}, {'player_unique': 'hassan-diarra-1', 'team_unique': 'connecticut', 'game_id': 430, 'game_date': '2025-03-23', 'game_time': '12:10:00', 'round_num': 3, 'competition_id': 6}, {'player_unique': 'samson-johnson-1', 'team_unique': 'connecticut', 'game_id': 430, 'game_date': '2025-03-23', 'game_time': '12:10

In [7]:
print(TODAY)

2024-04-06


In [5]:
def update_scores_from_csv(current_round: int, competition_id: int, game_scoring_csv = f"{datetime.date.today()}-game-scoring-2023-ncaa-tournament.csv"):
    player_game_insert = []
    losing_teams = []
    inactive_players = []
    team_1 = ""
    team_2 = ""
    with open(game_scoring_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        # next(reader)
        for row in reader:
            if row['lost'].upper() == 'L':
                losing_teams.append(row['team_unique'])
            if row['inactive'].upper() == 'I':
                inactive_players.append(row['player_unique'])
            if row['points'] != '':
                team_2 = team_1
                team_1 = row['team_unique']
                player_game_row = {
                    "player_unique": row['player_unique'],
                    "game_id": row['game_id'],
                    "points": row['points'],
                }
                player_game_insert.append(player_game_row)

    data = supabase.table("player_game").upsert(player_game_insert,
                                         ignore_duplicates=False, on_conflict="game_id, player_unique").execute()
    updated = supabase.table("competition_updated").insert({"competition_id": competition_id, "current_round": current_round}).execute()
    for team in losing_teams:
        loser_data = supabase.table('team_competition').update({'round_eliminated': current_round}).eq('team_unique', team).eq('competition_id', competition_id).execute()
        print("losers")
        print(loser_data)
    for player in inactive_players:
        player_data = supabase.table('player_competition').update({'inactive': True}).eq('player_unique', player).eq('competition_id', competition_id).execute()
        print("inactive players")
        print(player_data)
    print(data)

In [22]:
# date = datetime.date.today()
current_round = CURRENT_ROUND
# current_round=2
# date = TODAY
date = "2025-03-22"
update_scores_from_csv(current_round=current_round, competition_id=COMPETITION_ID, game_scoring_csv = f"{YEAR}/scores/{date}-game-scoring-{YEAR}-ncaa-tournament.csv")


losers
data=[{'team_unique': 'mcneese-state', 'competition_id': 6, 'league_unique': 'ncaambb', 'seed': 12, 'overall_seed': 49, 'region': 'MIDWEST', 'round_eliminated': 2, 'round_started': 2, 'team_stats': {'pace': 66.4, 'conference': 'southland', 'offensive_rating': 116.4, 'assist_percentage': 51.0, 'turnover_percentage': 13.9, 'strength_of_schedule': -2.29, 'free_throw_attempt_rate': 0.352, 'three_point_attempt_rate': 0.369, 'true_shooting_percentage': 0.574, 'effective_field_goal_percentage': 0.546, 'two_point_field_goal_percentage': 0.55, 'three_point_field_goal_percentage': 0.359, 'opp_effective_field_goal_percentage': 0.476}, 'stats_thru': '2025-03-16', 'team_win_loss': {'wins': 27.0, 'losses': 6.0, 'games_played': 0, 'conference_wins': 19.0, 'conference_losses': 1.0}}] count=None
losers
data=[{'team_unique': 'st-johns-ny', 'competition_id': 6, 'league_unique': 'ncaambb', 'seed': 2, 'overall_seed': 8, 'region': 'WEST', 'round_eliminated': 2, 'round_started': 2, 'team_stats': {'pac

In [64]:
print(CURRENT_ROUND)

3
